In [24]:
import json
import pennylane as qml
import pennylane.numpy as np

def create_Hamiltonian(h):
    """
    Function in charge of generating the Hamiltonian of the statement.

    Args:
        h (float): magnetic field strength

    Returns:
        (qml.Hamiltonian): Hamiltonian of the statement associated to h
    """

    # Put your code here #
    couplings = [-h for i in range(4)] + \
                [-1 for i in range(4)]
    
    ops = [qml.PauliX(i) for i in range(4)] + \
          [qml.PauliZ(i) @ qml.PauliZ((i+1)%4) for i in range(4)]

    return qml.Hamiltonian(couplings,ops)


dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def model(params, H):
    """
    To implement VQE you need an ansatz for the candidate ground state!
    Define here the VQE ansatz in terms of some parameters (params) that
    create the candidate ground state. These parameters will
    be optimized later.

    Args:
        params (numpy.array): parameters to be used in the variational circuit
        H (qml.Hamiltonian): Hamiltonian used to calculate the expected value

    Returns:
        (float): Expected value with respect to the Hamiltonian H
    """

    # Put your code here #
    for i in range(4):
        qml.RY(params[i], wires=i)
    
    for i in range(4):
        qml.CNOT([i,(i+1)%4])
    for i in range(4):
        qml.RZ(params[i+4], wires=i)
        
    for i in range(4):
        qml.RY(params[i+8], wires=i)
    
    for i in range(4):
        qml.CNOT([i,(i+1)%4])
    for i in range(4):
        qml.RZ(params[i+12], wires=i)

    return qml.expval(H)


def train(h):
    """
    In this function you must design a subroutine that returns the
    parameters that best approximate the ground state.

    Args:
        h (float): magnetic field strength

    Returns:
        (numpy.array): parameters that best approximate the ground state.
    """

    # Put your code here #
    H = create_Hamiltonian(h)
    theta = np.array([np.pi/2 for i in range(16)], requires_grad=True)
    
    opt = qml.GradientDescentOptimizer(stepsize=0.2)

    max_iterations = 500
    conv_tol = 1e-06

    for n in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(model, theta, H = H)
        energy = model(theta, H)

        if np.abs(energy - prev_energy) < conv_tol:
            print(f"Step = {n},  Energy = {energy:.8f} Ha")
            break
        
    return theta


# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    params = train(ins)
    return str(model(params, create_Hamiltonian(ins)))


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-1
    ), "The expected value is not correct."


test_cases = [['1.0', '-5.226251859505506'], ['2.3', '-9.66382463698038']]

for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '1.0'...
Step = 163,  Energy = -5.12420134 Ha
Correct!
Running test case 1 with input '2.3'...
Step = 299,  Energy = -9.63571670 Ha
Correct!
